In [1]:
# import necessary libraries

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import animation

%matplotlib notebook

# II. Reversible Cellular Automata

Our first notebook looked at Conway's "Game of Life" - the most famous example of a cellular automaton. The popularity of that model, and the fact that it is a bit simpler to implement than the next model we'll look at, make it a good place to start. But as we discussed, Game of Life possesses one feature which will likely be a problem as we move toward <i>quantum</i> automata. The Game of Life rules are not reversible. Given the initial state of the system at time $t=0$, we can predict the state at a later timestep $t$. But we cannot recover the initial state at time $t=0$ from the state at the future time $t$, because many initial states can evolve to the same final state. 

Quantum mechanics, on the other hand, is inherently *reversible*. A quantum state $|\psi(t)\rangle$ at time $t$ is obtained from an initial state $|\psi(0)\rangle$ as
$$
|\psi(t)\rangle = \hat{U}(t)|\psi(0)\rangle ,
$$
where the *time evolution operator* $\hat{U}(t)$ is *unitary*, meaning that its inverse is its Hermitian conjugate, $\hat{U}^{-1}(t) = \hat{U}^\dagger(t)$. This means that we can always formally recover the initial state from the state at $t$ via
$$
|\psi(0)\rangle = \hat{U}^\dagger(t)|\psi(t)\rangle.
$$

In order to more easily extend the physics of a cellular automaton to a quantum theory, we'll work with a <b>reversible</b> automaton going forward. In this notebook, we'll introduce a particular model called [Critters](https://en.wikipedia.org/wiki/Critters_(cellular_automaton%29), invented by Toffoli and Margolus in 1987, which we'll quantize in the next notebook.

## Critters

The <b>Critters</b> model shares many similarities with Game of Life. It consists of a square grid of cells which each can be `ON` or `OFF`, and are updated based upon what states their neighbors are currently in at each timestep.

The big difference in the Critters model is the ruleset and what we mean by "neighbors;" Critters is an example of a [block cellular automaton](https://en.wikipedia.org/wiki/Block_cellular_automaton), which defines a cell's neighbors using the [Margolus neighborhood](https://conwaylife.com/wiki/Margolus_neighbourhood). What we do in these kind of automata is first divide the grid into 2x2 "blocks," demarcated by the blue lines seen below in the figure (image credit: [Wikipedia Commons](https://en.wikipedia.org/wiki/File:Critters_transition_rule.svg)). Then, for each block, we update its state at the next timestep based on the rule in the figure, in which white cells are `OFF` and green cells are `ON`.

For example, looking at the top-left block in the figure, we see that, if a block is currently made up of four `OFF` cells, we must turn all four of those cells `ON` in the next timestep. The bottom-right block in the figure tells us that if a block currently contains four `ON` cells, we must turn them all `OFF` in the next timestep. The other possible configurations are defined similarly here.

Each distinct block state at the current timestep transitions into a distinct block state at the next timestep. This is what makes the Critters ruleset <b>reversible</b> - we can tell what the state of the block was at the previous timestep by looking at its current state.

Importantly, there is also one further subtlety. Every time step, we also alternate between dividing blocks using the blue lines in the figure, and the red dotted lines in the figure. In other words, the lines dividing up the 2x2 blocks shifts diagonally by one cell at each timestep. The rules however do not change for each individual block at each timestep.

![critters rules](img/critters_rules.svg)

<span style="background-color:orange"><b>Note:</b></span> The Critters ruleset illustrated above has one very peculiar behavior. A grid with only `OFF` cells in its initial state will evolve by continually flickering back and forth between `OFF` and `ON`. The way this is usually handled is to invert the image of the grid before displaying it, if an odd number of timesteps have elapsed.

A nice discussion of Critters is given by Margolus in sec. 1.4 of his paper [Crystalline Computation](https://arxiv.org/abs/comp-gas/9811002).

Let's now go ahead and implement some Critters! First, we'll translate the rules in the image above into a Python dictionary, where the keys will be the possible current states of a 2x2 block, and the associated values will be the states to which those current states transition, according to the image above. In this implementation, in contrast to what we did with Game of Life, it'll make our lives easier to use `-1`, rather than `0`, to represent `OFF`. `ON` will still be represented by `1`.

In [2]:
# define rules via lookup dicts

# first list the possible block states (left side of the image above)
current_states = [(-1,-1,-1,-1), (-1,-1,-1,1), (-1,-1,1,-1), (-1,-1,1,1),
                  (-1,1,-1,-1), (-1,1,-1,1), (-1,1,1,-1), (-1,1,1,1),
                  (1,-1,-1,-1), (1,-1,-1,1), (1,-1,1,-1), (1,-1,1,1),
                  (1,1,-1,-1), (1,1,-1,1), (1,1,1,-1), (1,1,1,1)]

# then list the states those states transition into (right side of the image)
next_states = [(1,1,1,1), (1,1,1,-1), (1,1,-1,1), (-1,-1,1,1),
               (1,-1,1,1), (-1,1,-1,1), (-1,1,1,-1), (-1,-1,-1,1),
               (-1,1,1,1), (1,-1,-1,1), (1,-1,1,-1), (-1,-1,1,-1),
               (1,1,-1,-1), (-1,1,-1,-1), (1,-1,-1,-1), (-1,-1,-1,-1)]

# put these together into a dictionary
rules = {}
for i,s in enumerate(current_states):
    rules[s] = next_states[i]

Next we'll define some constants, like the size of our grid and the colors we'll use

In [3]:
# define constants

GRID_SIZE = 100      # we'll use a (100 cell) x (100 cell) grid
SEED = 44            # RNG seed for reproducible random initial data

# colormap for animations
# dead cells will be black, live cells will be green
cmap = ListedColormap(["black", "lime"])

We'll initialize the grid by picking a rectangular chunk of the grid in which we'll randomly turn `ON` some cells:

In [4]:
# initialize grid

# seed RNG (optional, used for reproducible results)
np.random.seed(SEED)

# critters initial conditions
grid = -1*np.ones((GRID_SIZE, GRID_SIZE))
grid[70:80,70:80] = np.random.choice([-1,1], (10,10), p=[0.4,0.6])

The cell below can be run to see what results from this initial "blob" of `ON` cells.

In [ ]:
# animate

# initialize figure
fig = plt.figure(figsize=(6,6))
ax = fig.gca()
ax.axis('off')
im = ax.imshow(grid, cmap=cmap)

even_blocks = [[i,j] for i in range(0,GRID_SIZE,2) for j in range(0,GRID_SIZE,2)]
odd_blocks = [[i,j] for i in range(1,GRID_SIZE,2) for j in range(1,GRID_SIZE,2)]

# define update function
def critters_update(n):
    global grid
    
    # `mult` is -1 for odd blocks, +1 for even ones
    # used for the common practice of inverting the image before displaying it, after odd updates
    # (inverting is multiplying by -1 in our representation)
    # to avoid flickering of "empty space" cells
    mult = 1 
    blocks = None
    
    if (n%2):
        blocks = odd_blocks
        mult = -1
    else:
        blocks = even_blocks
        
    for i,j in blocks:
        block = (grid[i,j], grid[i,(j+1)%GRID_SIZE],
                 grid[(i+1)%GRID_SIZE,j], grid[(i+1)%GRID_SIZE,(j+1)%GRID_SIZE])

        a,b,c,d = rules[block]
        
        grid[i,j] = a
        grid[i,(j+1)%GRID_SIZE] = b
        grid[(i+1)%GRID_SIZE,j] = c
        grid[(i+1)%GRID_SIZE,(j+1)%GRID_SIZE] = d
            
    # im.set_data(mult*grid)
    return ax.imshow(mult*grid, cmap=cmap)

frames = 100

ims = [(im,)]

for n in range(1,frames): 
    ims.append((critters_update(n),))
        
anim = animation.ArtistAnimation(fig, ims, interval=88, repeat_delay=0, blit=True)
# anim.save("img/glider_scattering_2.gif", writer="imagemagick")

This is what happened when the cell above was run for the first time, and the output saved as a .gif:

![critters](img/critters.gif)

The "blob of stuff" that we initialized the grid with kind of undulates a bit, and then starts spitting out "gliders" after a while:

![glider](img/glider.gif)

## Glider Scattering

The gliders existing in the Critters world will be our main focus when we go to the quantum case. In particular, they can exhibit some very interesting "scattering" behavior that we'll make use of.

Below are two nearly-identical initial conditions, each involving two gliders each. Below these images are simulations of how these initial conditions evolve.

Scattering 1 | Scattering 2
:- | :- 
![gliders_init_1](img/glider_scattering_1.png) | ![gliders_init_2](img/glider_scattering_2.png)
![glider_scattering_1](img/glider_scattering_1.gif) | ![glider_scattering_2](img/glider_scattering_2.gif)

In both cases, the gliders approach each other, interact, and then go their separate ways. Although the initial conditions used in each case are almost identical (see below), it is clear that the outcomes of these two events are very different. In the first case, the gliders head toward each other horizontally, but leave the interaction traveling vertically. In the second, they again approach each other horizontally but end up basically passing right through one another.

Below are the initial conditions used in each of these scattering setups. The only difference is that the gliders are 2 cells closer together in the second scenario than they are in the first.

In [ ]:
'''
# Scattering 1

grid = -1*np.ones((GRID_SIZE, GRID_SIZE))

grid[40, 40] = 1
grid[41, 41] = 1
grid[42, 41] = 1
grid[43, 40] = 1

grid[40, 59] = 1
grid[41, 58] = 1
grid[42, 58] = 1
grid[43, 59] = 1


# Scattering 2

grid = -1*np.ones((GRID_SIZE, GRID_SIZE))

grid[40, 42] = 1
grid[41, 43] = 1
grid[42, 43] = 1
grid[43, 42] = 1

grid[40, 59] = 1
grid[41, 58] = 1
grid[42, 58] = 1
grid[43, 59] = 1
'''

## Conclusions

We've seen here yet another model cellular automaton called Critters. This model exhibits emergent behavior which is similar in complexity to the Game of Life, with the added feature of being governed by "physics" which is time-symmetric (reversible). This feature makes it incredibly simple to port over to a quantum computer. 

In the next notebook, that's just what we'll do. We'll discuss how to implement the Critters ruleset using [Qiskit](https://qiskit.org/), and by the end of the notebook we'll be able to witness a "scattering" event like those simulated here - only next time we'll be doing the scattering using a glider in a <b>quantum superposition</b>! 